In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/heart-disease-prediction-using-logistic-regression/framingham.csv


In [2]:
df = pd.read_csv('/kaggle/input/heart-disease-prediction-using-logistic-regression/framingham.csv')

In [4]:
df.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [5]:
from keras.models import Sequential
from keras.layers import Dense



Using TensorFlow backend.


In [48]:
model = Sequential()
model.add(Dense(units=64,activation='relu', input_dim= 15))
model.add(Dense(units=64, activation='relu', input_dim=100))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='sgd', metrics=['accuracy'])

In [7]:
y = df["TenYearCHD"]
y

0       0
1       0
2       0
3       1
4       0
       ..
4233    1
4234    0
4235    0
4236    0
4237    0
Name: TenYearCHD, Length: 4238, dtype: int64

In [25]:

X = df[df.columns[:-1]]

X

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4233,1,50,1.0,1,1.0,0.0,0,1,0,313.0,179.0,92.0,25.97,66.0,86.0
4234,1,51,3.0,1,43.0,0.0,0,0,0,207.0,126.5,80.0,19.71,65.0,68.0
4235,0,48,2.0,1,20.0,NaN,0,0,0,248.0,131.0,72.0,22.00,84.0,86.0
4236,0,44,1.0,1,15.0,0.0,0,0,0,210.0,126.5,87.0,19.16,86.0,NaN


In [26]:
from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values='NaN', strategy='mean', axis=0)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [36]:
transformed_values = imputer.fit_transform(X)
transformed_values

array([[  1.        ,  39.        ,   4.        , ...,  26.97      ,
         80.        ,  77.        ],
       [  0.        ,  46.        ,   2.        , ...,  28.73      ,
         95.        ,  76.        ],
       [  1.        ,  48.        ,   1.        , ...,  25.34      ,
         75.        ,  70.        ],
       ...,
       [  0.        ,  48.        ,   2.        , ...,  22.        ,
         84.        ,  86.        ],
       [  0.        ,  44.        ,   1.        , ...,  19.16      ,
         86.        ,  81.96675325],
       [  0.        ,  52.        ,   2.        , ...,  21.47      ,
         80.        , 107.        ]])

In [52]:
from sklearn.model_selection import train_test_split
X_train,x_test,y_train,y_test = train_test_split(transformed_values,y, test_size = 0.25, random_state = 42, shuffle = True)

In [53]:
model.fit(X_train,y_train, epochs = 150)

Epoch 1/150
3178/3178 [==============================] - 0s 67us/step - loss: 0.4128 - accuracy: 0.8474
Epoch 2/150
3178/3178 [==============================] - 0s 46us/step - loss: 0.4129 - accuracy: 0.8446
Epoch 3/150
3178/3178 [==============================] - 0s 45us/step - loss: 0.4141 - accuracy: 0.8439
Epoch 4/150
3178/3178 [==============================] - 0s 45us/step - loss: 0.4100 - accuracy: 0.8452
Epoch 5/150
3178/3178 [==============================] - 0s 47us/step - loss: 0.4114 - accuracy: 0.8449
Epoch 6/150
3178/3178 [==============================] - 0s 46us/step - loss: 0.4129 - accuracy: 0.8464
Epoch 7/150
3178/3178 [==============================] - 0s 46us/step - loss: 0.4120 - accuracy: 0.8458
Epoch 8/150
3178/3178 [==============================] - 0s 44us/step - loss: 0.4126 - accuracy: 0.8464
Epoch 9/150
3178/3178 [==============================] - 0s 44us/step - loss: 0.4133 - accuracy: 0.8471
Epoch 10/150
3178/3178 [==============================] - 0s 44u

3178/3178 [==============================] - 0s 45us/step - loss: 0.4093 - accuracy: 0.8455
Epoch 80/150
3178/3178 [==============================] - 0s 46us/step - loss: 0.4094 - accuracy: 0.8464
Epoch 81/150
3178/3178 [==============================] - 0s 46us/step - loss: 0.4098 - accuracy: 0.8468
Epoch 82/150
3178/3178 [==============================] - 0s 46us/step - loss: 0.4080 - accuracy: 0.8458
Epoch 83/150
3178/3178 [==============================] - 0s 46us/step - loss: 0.4100 - accuracy: 0.8496
Epoch 84/150
3178/3178 [==============================] - 0s 48us/step - loss: 0.4080 - accuracy: 0.8468
Epoch 85/150
3178/3178 [==============================] - 0s 46us/step - loss: 0.4087 - accuracy: 0.8442
Epoch 86/150
3178/3178 [==============================] - 0s 44us/step - loss: 0.4099 - accuracy: 0.8480
Epoch 87/150
3178/3178 [==============================] - 0s 46us/step - loss: 0.4058 - accuracy: 0.8455
Epoch 88/150
3178/3178 [==============================] - 0s 46us/st

In [57]:
_,acc = model.evaluate(x_test,y_test)
print("Acc:{}",acc*100)

1060/1060 [==============================] - 0s 65us/step
Acc:{} 85.75471639633179
